In [4]:
from data import dataset, PlantOrgansDataset
from preprocessing import preprocess_image_and_mask
import torchvision.transforms.v2 as T
import torch
import numpy as np
from alexnet import patch_index_to_position, image_to_patches, MyTransform
from train import device, pixel_validate, patch_loss, patch_validate, evaluate, fit

In [5]:
commonTransform = T.Compose([
        T.Resize(size=(2048, 2048)),
        T.ToImage()
        
        # T.RandomHorizontalFlip(p=0.5),
        # T.RandomVerticalFlip(p=0.5),
        # T.RandomRotation(degrees=45)
    ])
imagesTransform = T.Compose([
    T.ToDtype(torch.float32, scale=False),
    # T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    MyTransform(64),
    T.Resize((224, 224)),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
masksTransform = T.Compose([
    T.ToDtype(torch.int8, scale=False),
    # T.Normalize(mean=[0.0014], std=[0.0031]),
    MyTransform(64),
    # T.Resize((224, 224))
])

In [6]:
train_validation_data = dataset['train'].train_test_split(test_size=0.2, seed=42)
train_dataset = PlantOrgansDataset(train_validation_data['train'], commonTransform, imagesTransform, masksTransform)
validation_dataset = PlantOrgansDataset(train_validation_data['test'], commonTransform, imagesTransform, masksTransform)
test_dataset = PlantOrgansDataset(dataset['validation'], commonTransform, imagesTransform, masksTransform)


In [7]:
tr = T.Compose([
    T.ToImage(),
    T.ToDtype(dtype=torch.int8, scale=False)
])
tensor = tr(train_validation_data['train'][0]['label'])
tensor.max()

tensor(4, dtype=torch.int8)

In [8]:
cross_entropy_weights = torch.tensor([
        4.8033e-04,
        6.4129e-03,
        3.9272e-03,
        9.7140e-01,
        1.7778e-02], device=device)

In [9]:
print("train_dataset: ", len(train_dataset))
print("validation_dataset: ", len(validation_dataset))
print("test_dataset: ", len(test_dataset))

train_dataset:  4596
validation_dataset:  1149
test_dataset:  1437


In [10]:
class WrappedDataLoader:
    def __init__(self, loader, func):
        self.loader = loader
        self.func = func

    def __len__(self):
        return len(self.loader)

    def __iter__(self):
        for batch in iter(self.loader):
            batch_cuda = []
            for X, y in batch:
                batch_cuda.append(self.func(X, y))
            yield batch_cuda

In [11]:
def to_device(X: torch.Tensor, y: torch.Tensor):
    return X.to(device, dtype=torch.float32), y.to(device, dtype=torch.int8)

In [12]:
batch_size = 1024

In [13]:
def custom_collate_fn(batch):
    batchs_amount = len(batch)
    current_images = []
    current_masks = []
    current_length = 0
    i = 0
    while i < batchs_amount or current_length >= batch_size:
        if current_length == batch_size:
            if len(current_images) == 1:
                result_images = current_images[0]
                result_masks = current_masks[0]
            else:
                result_images = torch.concatenate(current_images)
                result_masks = torch.concatenate(current_masks)
            current_images = []
            current_masks = []
            current_length = 0
            yield result_images, result_masks
        elif current_length > batch_size:
            concatenated_images = torch.concatenate(current_images)
            concatenated_masks = torch.concatenate(current_masks)
            images_split = torch.split(concatenated_images, batch_size, dim=0)
            masks_split = torch.split(concatenated_masks, batch_size, dim=0)
            current_images = [images_split[len(images_split) - 1]]
            current_masks = [masks_split[len(masks_split) - 1]]
            current_length = len(current_images[0])
            for j in range(len(images_split) - 1):
                yield images_split[j], masks_split[j]
        else:  
            images, masks = batch[i]
            i += 1
            current_length += len(images)
            current_images.append(images)
            current_masks.append(masks)
    if current_length > 0:
        concatenated_images = torch.concatenate(current_images)
        concatenated_masks = torch.concatenate(current_masks)
        yield concatenated_images, concatenated_masks



In [14]:
from torch.utils.data import DataLoader
train_loader = WrappedDataLoader(
    DataLoader(train_dataset, batch_size=1, shuffle=False, collate_fn=custom_collate_fn, 
               pin_memory=False, pin_memory_device=[device]), to_device)
valid_loader = WrappedDataLoader(
    DataLoader(validation_dataset, batch_size=1, shuffle=False, collate_fn=custom_collate_fn,
               pin_memory=True, pin_memory_device=[device]), to_device)
test_loader = WrappedDataLoader(
    DataLoader(test_dataset, batch_size=1, shuffle=False, collate_fn=custom_collate_fn,
               pin_memory=True, pin_memory_device=[device]), to_device)

In [15]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=False).to(device)

model.classifier[6] = torch.nn.Linear(in_features=4096, out_features=5, bias=True).to(device)

# model = AlexNet([11, 5, 3, 3, 3], [96, 256, 384, 384, 256], [0, 2, 1, 1, 1], [4096, 4096], 4, 224).to(device)

Using cache found in C:\Users\pc/.cache\torch\hub\pytorch_vision_v0.10.0
c:\ProgramData\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\ProgramData\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [16]:
to_image = T.ToPILImage()
mask_to_image = T.Compose([
    T.ToTensor(),
    T.ToDtype(torch.float16),
    T.Normalize(mean=[0.0014], std=[0.0031]),
    T.ToPILImage(),
    
])

c:\ProgramData\anaconda3\Lib\site-packages\torchvision\transforms\v2\_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [17]:
# for batch in valid_loader:
#     # print(len(batch))
#     for X, y in batch:
#         print(X.shape)
#         predicted = model(X)
#         loss_, correct, _ = patch_validate(model, torch.nn.CrossEntropyLoss(weight=cross_entropy_weights), X, y)
#         print(loss_, correct)
#         loss_, correct, _ = pixel_validate(model, torch.nn.CrossEntropyLoss(weight=cross_entropy_weights), X, y)
#         print(loss_, correct)
#     break

In [18]:
# torch.cuda.empty_cache()

In [19]:
# counts = torch.zeros(size=(5, 1), dtype=torch.int64).to(device)
# for batch in valid_loader:
#     for X, y in batch:
#         current_counts = y.unique(return_counts=True)
#         for i in current_counts[0]:
#             counts[current_counts[0][i]] += current_counts[1][i]
    
            

In [20]:
# counts_sum = torch.sum(counts).to(device)

In [21]:
# weights = torch.div(counts_sum, torch.multiply(counts, 2549.76513671875))

# print(weights)

In [22]:
import os
import torch.optim as optim
import time
from ray import tune
from ray.train import Checkpoint, get_checkpoint, report, RunConfig
from ray.tune.schedulers import ASHAScheduler

2024-11-15 19:52:49,281	INFO util.py:154 -- Outdated packages:
  ipywidgets==7.8.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-11-15 19:52:49,623	INFO util.py:154 -- Outdated packages:
  ipywidgets==7.8.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [ ]:
src_path = "C:\\Users\\pc\\Documents\\repos\\mp-2\\nn\\nn-lab2\\"

constants = {
    "criterion": torch.nn.CrossEntropyLoss(),
    "lr": 0.0001,
    "n_epochs": 40,
    "saving_model_path": src_path + "models\\raytune"
}
config = {
    "batch_size": tune.grid_search([64*64]),
    "patch_size": tune.grid_search([32])
}

In [24]:
def train(config, constants):
    # model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=False).to(device)
    # model.classifier[6] = torch.nn.Linear(in_features=4096, out_features=4, bias=True).to(device)

    criterion = constants["criterion"]
    optimizer = optim.Adam(model.parameters(), lr=constants["lr"])
    n_epochs = constants["n_epochs"]
    saving_model_path = os.path.join(constants["saving_model_path"], 
                                "checkpoint_{batch_size}_{patch_size}.model".format(
                                    batch_size=config['batch_size'], 
                                    patch_size=config['patch_size']))

    # train_loader = WrappedDataLoader(DataLoader(train_dataset, batch_size=1, shuffle=True), to_device)
    # valid_loader = WrappedDataLoader(DataLoader(validation_dataset, batch_size=1, shuffle=False), to_device)
    # test_loader = WrappedDataLoader(DataLoader(test_dataset, batch_size=1, shuffle=False), to_device)

    print('\nFitting nn model')
    start_time = time.time()

    losses_arr = fit(n_epochs, model, criterion, optimizer, test_loader, valid_loader, cross_entropy_weights.to(device))
    print(f'Fit time: {time.time() - start_time} s')

    check_point = torch.load('final_model.pt', map_location=device)
    model.load_state_dict(check_point)

    test_loss, test_accuracy = evaluate(model, criterion, test_loader)

    if saving_model_path is not None:
        print('Saving model')
        torch.save((model.state_dict(), optimizer.state_dict()), saving_model_path)
        checkpoint = Checkpoint.from_directory(constants["saving_model_path"])
        report(
            {"loss": test_loss, "accuracy": test_accuracy},
            checkpoint = checkpoint
        )


In [25]:
def train_model(num_samples=2, gpus_per_trial=0.125):
    tuner = tune.Tuner(
        tune.with_resources(
            tune.with_parameters(train, constants=constants),
            resources={"cpu": 0.25, "gpu": gpus_per_trial}
        ),
        tune_config=tune.TuneConfig(
            metric="accuracy",
            mode="max",
            # scheduler=scheduler,
            num_samples=num_samples,
            # search_alg=ax_search
        ),
        param_space=config,
        run_config=RunConfig(storage_path=os.path.join(src_path, "raytune"))
    )
    results = tuner.fit()
    
    best_result = results.get_best_result("accuracy", "max")

    return best_result

In [26]:
# best = train_model(num_samples=1)
result = train({"batch_size": 1024, "patch_size": 64}, constants)


Fitting nn model


100%|██████████| 1149/1149 [1:13:22<00:00,  3.83s/it]



epoch:   1, loss: 3.73238, valid loss: 0.66965, valid accruacy: 81.651%
Validation loss decreased (inf --> 0.669654). Saving model...


  2%|▏         | 25/1437 [01:27<1:22:35,  3.51s/it]


KeyboardInterrupt: 

In [27]:
torch.cuda.empty_cache()

check_point = torch.load('final_model.pt', map_location=device)
model.load_state_dict(check_point)
# test_loss, test_accuracy = evaluate(model, constants["criterion"], test_loader)

C:\Users\pc\AppData\Local\Temp\ipykernel_10928\4026949323.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  check_point = torch.load('final_model.pt', map_location=device)

<All keys matched successfully>

In [ ]:
model.eval()
for batch in valid_loader:
    for X, y in batch:
        predicted = model(X)
        print(torch.argmax(predicted, dim=1).unique(return_counts=True)[0].item(), " actual ", y.unique(return_counts=True))

    

In [36]:
model.parameters()()

TypeError: 'generator' object is not callable